 ## Create the data

In [1]:
# import required modules
from mimesis.locales import Locale
from mimesis.keys import maybe
from mimesis.schema import Field, Schema
from mimesis import Datetime

dt = Datetime(seed=42)
f = Field(locale=Locale.EN_GB, seed=42)

from random import Random
from uuid import UUID, uuid4

rnd = Random()
rnd.seed(10)

table_definition = {
    "person":{
        "amount":1000
    },
    "product":{
        "amount":1000
    },
    "order":{
        "amount":10000
    },
    "artist":{
        "amount":500
    },
    "review":{
        "amount":2000
    }
}

# create data

## person

def person_generator() -> dict:
    first_name = f('first_name')
    last_name = f('last_name')
    return {
        "_id":UUID(int=rnd.getrandbits(128), version=4),
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.9)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

person_schema = Schema(
    schema=person_generator,
    iterations=table_definition['person']['amount']
)
person_data = person_schema.create()

person_id_count = range(table_definition['person']['amount']-1)

print("person data created")

## artist

def artist_generator() -> dict:
    first_name = f("first_name")
    last_name = f("last_name")
    return {
        "_id":f('increment'), 
        "first_name":first_name,
        "last_name":last_name,
        "name":first_name + " " + last_name,
        "company_name":f("company", key=maybe(None, probability=0.5)),
        "email":f("email"),
        "phone":f("phone_number"),
        "address":{
            "address_line_1":f("street_number")+" "+f("street_name"),
            "address_line_2":f('choice', items=['apt. 10','Suite. 23'], key=maybe(None, probability=0.9)),
            "city":f("city"),
            "country":f('choice', items=['England','Scotland', 'Wales', 'Northern Ireland']),
            "post_code":f("postal_code"),
            "coordinates":[f('latitude'), f('longitude')]
        }
    }

artist_schema = Schema(
    schema=artist_generator,
    iterations=table_definition['artist']['amount']
)

artist_data = artist_schema.create()

artist_id_count = range(table_definition['artist']['amount']-1)

print("artist data created")

## product

def product_generator() -> dict:
    created_at = dt.datetime(start=2023, end=2023)
    quantity = rnd.randint(1, 20)
    return {
        "_id":UUID(int=rnd.getrandbits(128), version=4),
        "name":' '.join(f('words', quantity=2)),
        "description":' '.join(f('words', quantity=rnd.randint(8, 25))),
        "category":f('choice', items=["oil paint", "watercolor", "acrylic paint", "charcoal", "pencil", "ink", "pastel", "collage", "digital art", "mixed media"]),
        "price":f('price', minimum=500, maximum=25000),
        "currency":f('currency_symbol'),
        "discount":f('float_number', start=0.2, end=0.8, precision=1, key=maybe(None, probability=0.8)),
        "quantity":quantity, 
        "image_url":f('stock_image_url'),
        "artist":artist_data[f('choice', items=artist_id_count)]['_id'],
        "creation_history": {
            "created_at":created_at,
            "quantity":quantity
        }
    }

product_schema = Schema(
    schema=product_generator,
    iterations=table_definition['product']['amount']
)

product_data = product_schema.create()

product_id_count = range(table_definition['product']['amount']-1)

print("product data created")

## order

def order_generator() -> dict:
    person_number = f('choice', items=person_id_count)
    product_number = f('choice', items=product_id_count)
    shipping_address = person_data[person_number]['address']
    order_date = dt.datetime(start=2023, end=2023)
    return {
        "_id":UUID(int=rnd.getrandbits(128), version=4),
        "person":person_data[person_number]['_id'],
        "product":product_data[product_number]['_id'],
        "product_name":product_data[product_number]['name'],
        "currency":product_data[product_number]['currency'],
        "discount":product_data[product_number]['discount'],
        "price":product_data[product_number]['price'],
        "quantity": rnd.randint(1, 3),
        "order_date":order_date,
        "shipping_address":shipping_address,
        "payment_method":f('choice', items=['credit card','debit card', 'PayPal']),
        "order_status":f('choice', items=['pending','processing', 'shipped', 'delivered'], key=maybe(None, probability=0.1))
    }

order_schema = Schema(
    schema=order_generator,
    iterations=table_definition['order']['amount']
)

order_data = order_schema.create()

print("order data created")

## review
def review_generator() -> dict:
    return {
        "_id":UUID(int=rnd.getrandbits(128), version=4),
        "person":person_data[f('choice', items=person_id_count)]['_id'],
        "product":product_data[f('choice', items=product_id_count)]['_id'],
        "artist":artist_data[f('choice', items=artist_id_count)]['_id'],
        "rating":f('choice', items=[1,2,3,4,5]),
        "review_text":' '.join(f('words', quantity=rnd.randint(8, 50)))
    }

review_schema = Schema(
    schema=review_generator,
    iterations=table_definition['review']['amount']
)

review_data = review_schema.create()

print("review data created")


person data created
artist data created
product data created
order data created
review data created


In [2]:
[product_data[i]['artist'] for i in range(5)]
# [83, 281, 174, 315, 361]

[83, 281, 174, 315, 361]

 ## Load the data

In [3]:
from pymongo import MongoClient
from bson.binary import UuidRepresentation
from datetime import datetime,timezone

MONGODB_URI = "mongodb://localhost:27017/"

client = MongoClient(MONGODB_URI, uuidRepresentation='standard')
db = client["surreal_deal"]

# Create collections
person = db["person"]
product = db["product"]
order = db["order"]
artist = db["artist"]
review = db["review"]


In [4]:
%%time
person.insert_many(person_data)


CPU times: user 6.19 ms, sys: 2.17 ms, total: 8.36 ms
Wall time: 44.5 ms


InsertManyResult([UUID('7b89296c-6dcb-4c50-8857-7eb1924770d3'), UUID('766bad07-34c2-4a80-83cc-0f2793fdcab8'), UUID('470b9805-d2d6-4877-bdc5-9a3ad035d259'), UUID('08ceac39-2904-4def-8f84-b683a749f9c5'), UUID('137a9777-53e8-4b43-bd76-3fb9854a9657'), UUID('bedc25e6-f3eb-4f12-b3d0-6f863fffc830'), UUID('6b9f15c4-0b68-4c1c-9c74-e45eff1e5bef'), UUID('5ae6a228-9a6a-4329-a381-23e5dc338383'), UUID('d3ac535f-489b-440f-abd7-f50361b0ee09'), UUID('2cb7362c-74f2-42ed-8327-79eeacca7f0d'), UUID('5cd2875e-a96e-42b3-8d98-4bffaf949e5e'), UUID('dc2c2e2c-c491-44d0-b4f9-42cb220adb0a'), UUID('708cc1b6-f829-429f-bd48-06c2fb7f6f5d'), UUID('953b00b0-0b54-4a22-a00f-ecc19d02fc90'), UUID('31e875ba-224c-4601-bc53-d0e30109c207'), UUID('894deab4-4d88-450f-a8da-c663f0e58650'), UUID('3d7c95f9-e5f0-407e-85a5-6d7e5dbbb7ce'), UUID('735c076b-8c8a-48b2-aaac-3142507a2560'), UUID('a7298c66-10a3-4558-b850-36de6f9fb997'), UUID('807da245-d814-4575-931e-c56c95a4d257'), UUID('f843bdb8-396b-483a-9798-c9cf280b11fd'), UUID('08208d09-0

In [5]:
%%time
product.insert_many(product_data)


CPU times: user 4.21 ms, sys: 1.1 ms, total: 5.31 ms
Wall time: 34.5 ms


InsertManyResult([UUID('5ce4e36b-bbae-473c-a3be-902a9af1977d'), UUID('7d074582-838a-4766-b160-8a98060a8abb'), UUID('bdde5f7d-9b48-447a-97c2-aa5b4a914554'), UUID('82d4e478-e3d3-4180-be4d-06cbbc56d5c4'), UUID('01ccaa73-9925-4a53-b379-41c5b1762209'), UUID('65a7457e-0a0a-4b40-be64-4d7da8d01d7a'), UUID('326dc4f6-c92c-4264-8ddf-58531a4bda90'), UUID('677cdaef-8bbb-4ff0-bcb5-21ab57961be7'), UUID('e8a53eb6-e687-4a9e-94ab-202ba6c291b9'), UUID('213621a6-7522-49f7-afeb-0fdabfd72255'), UUID('2ee082f1-adaa-44a9-ad5d-77b3452745ce'), UUID('df081856-cc70-485d-9849-c905912f122d'), UUID('ecb83aef-4ea7-49f0-84da-d98d26ced1eb'), UUID('81515523-ed15-47cf-b160-99bc9e9a16b4'), UUID('7c54bf53-4611-44cc-bca3-bec0fe7b5c88'), UUID('3983789f-e5dd-47eb-9f81-e88987ad68c6'), UUID('e8771cd8-0514-4513-8799-6b1cb2ec934e'), UUID('44680b43-ede7-44b5-920d-377cbb27fe7f'), UUID('e2d1a198-1886-4f58-b07c-66611ae50d55'), UUID('d0ce10f7-b93c-4f85-9d85-6310a5308c81'), UUID('b6787d9e-2cdd-4b21-b02e-2ba2509d09b0'), UUID('3dcbc1c2-3

In [6]:
%%time
order.insert_many(order_data)


CPU times: user 47.9 ms, sys: 3.88 ms, total: 51.7 ms
Wall time: 131 ms


InsertManyResult([UUID('8513a099-6641-45b2-ab8e-645bd03eebd8'), UUID('27d2e7f1-09c0-4e2e-9aec-723287d54c24'), UUID('b2eacb42-75c0-4054-8131-3d8af4d2f68d'), UUID('3931acf3-e540-428a-bbae-2ed5be8f3958'), UUID('11700af3-5a78-4daf-a8b8-854ddad91ed0'), UUID('546ba762-c053-4e46-96e2-f9ae6bb94b0a'), UUID('b9037d5c-d861-4251-b18c-11078260ea5a'), UUID('7f0e0ad9-9e8b-41e8-8158-6c2dc7bfa35c'), UUID('3abcb2a2-6bd7-4480-9ef4-fbd08e87046e'), UUID('3932e23f-dc17-452a-875e-91f56d2c6574'), UUID('cb480f3a-e5fe-4ce5-8638-3df9cd6c563a'), UUID('ab03a0f3-c4ec-4edc-b167-7a586784e7d5'), UUID('35d82e1e-0ec8-4cf6-8007-a76c0373f65a'), UUID('76858f8c-6370-4804-9b01-4b86c7328ba1'), UUID('d239b29e-62d2-45e7-969e-f447d2580832'), UUID('13d691ad-6c67-4695-9e33-c966b2da5192'), UUID('0d7f23df-6cbf-44f4-93a9-936da6c70283'), UUID('946c01d8-6d16-45cc-abd0-66ffb88fb490'), UUID('1d93d3dd-c99d-4e47-b10a-f1c5cbe9a112'), UUID('8c058f2d-77d6-49c5-86fd-13713db49bd7'), UUID('e71de9c2-8ebd-4c2f-92cd-a12cb99b29f5'), UUID('c2ec5fdd-5

In [7]:
%%time
artist.insert_many(artist_data)


CPU times: user 2.14 ms, sys: 996 µs, total: 3.13 ms
Wall time: 29.9 ms


InsertManyResult([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [8]:
%%time
review.insert_many(review_data)


CPU times: user 10.7 ms, sys: 1.44 ms, total: 12.1 ms
Wall time: 44.7 ms


InsertManyResult([UUID('3b8d2b28-f6b8-4cf5-b5b6-ae401b57bde0'), UUID('bac4f666-db26-430c-a6fd-906fec41b7f0'), UUID('8265ad22-edf4-4bdd-9661-3511cb9d0e16'), UUID('3d752240-da6c-4533-872e-468219739774'), UUID('d0152cc9-2d7c-494b-975f-aba3b1bf2a81'), UUID('075a17f2-7a27-4e3f-9dc8-735ea77b010d'), UUID('bd549e11-5aed-4591-9f8a-72dad7db4c43'), UUID('f8a0a3f8-3bfb-4779-832b-b325f4169f00'), UUID('fbe21c49-ab84-4a7d-a7a9-fe6d26fb9c9b'), UUID('c4ca3b20-9803-48fb-b658-a8132e0092bf'), UUID('0f5ba9f7-10fa-4aca-b628-48cbfa8b9e03'), UUID('164edfe8-71d8-4200-bd83-72a2fd977dad'), UUID('06e1f586-c920-421d-a262-2528caa37b2d'), UUID('ac9263a6-f1c8-4bb5-88a1-5a84abf78b00'), UUID('e6b53540-fd24-4eb9-b470-5e2cffaf605f'), UUID('b44115a5-7f4a-411e-b9bf-8b340c32c960'), UUID('4a104623-9bbe-48af-bc28-1d91d08ba271'), UUID('4929979a-c814-4026-b43b-e302df890bc1'), UUID('f639cf56-13ba-46fc-945c-2eeaadf58d81'), UUID('54b5ccb8-cf64-4fb3-94fc-b891d3e793dc'), UUID('91bcf51f-507e-40eb-8765-cfa09399e6ae'), UUID('77ca00af-e

 ## Run the queries

In [ ]:
from random import randint
# getting just an array of ids to use for loops in some queries
person_ids = person.distinct("_id")
product_ids = product.distinct("_id")
order_ids = order.distinct("_id")
artist_ids = artist.distinct("_id")
review_ids = review.distinct("_id")

# data too big error: Executor error during distinct command :: caused by :: distinct too big, 16mb cap, full error: {'ok': 0.0, 'errmsg': 'Executor error during distinct command :: caused by :: distinct too big, 16mb cap', 'code': 17217, 'codeName': 'Location17217'}


 ### Q1: lookup vs record links

In [11]:
# %%timeit
list(review.aggregate([
	{
		"$lookup": {
			"from": "artist",
			"localField": "artist",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "artist",
		}

	},
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}

	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "category": 1, "price": 1 } }
			],
			"as": "product",
		}
	}
]))


[{'_id': UUID('3b8d2b28-f6b8-4cf5-b5b6-ae401b57bde0'),
  'person': [{'name': 'Jeffrey Burns',
    'email': 'mount1823@outlook.com',
    'phone': '0999 674 7777'}],
  'product': [{'name': 'surf seeking',
    'category': 'digital art',
    'price': 21661.39}],
  'artist': [{'name': 'Dusty White',
    'email': 'collectables2034@outlook.com',
    'phone': '016977 4034'}],
  'rating': 4,
  'review_text': 'lawrence reporter stack broadcast basin describes women kings man dr tourism mom teach aging seat percentage subscribe spend mm sports solo inclusive monday frequent chapters strings travel mr ct scheme behalf slowly'},
 {'_id': UUID('bac4f666-db26-430c-a6fd-906fec41b7f0'),
  'person': [{'name': 'Eli Holden',
    'email': 'dash2073@example.com',
    'phone': '0800 355360'}],
  'product': [{'name': 'graphic previous',
    'category': 'digital art',
    'price': 2022.05}],
  'artist': [{'name': 'Yahaira Holcomb',
    'email': 'hash2092@outlook.com',
    'phone': '0310 731 9676'}],
  'rating'

 ### Q2: lookup vs graph

In [12]:
# %%timeit
list(order.aggregate([
	{
		"$lookup": {
			"from": "person",
			"localField": "person",
			"foreignField": "_id",
			"pipeline": [
				{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
			],
			"as": "person",
		}
	},
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "name": 1, "email": 1, "phone": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "category": 1, "description": 1, "image_url": 1, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{ "$project": { "_id": 0, "price": 1, "order_date": 1, "product_name": 1, "artist": 1, "person": 1, "product": 1 } }
]))


[{'person': [{'name': 'Dacia Crawford',
    'email': 'head2051@example.org',
    'phone': '0117 098 5739'}],
  'product': [{'description': 'since bookmark vitamin overall slots progressive icon reason walking dog seven malaysia juice icons re',
    'category': 'collage',
    'image_url': 'https://source.unsplash.com/1920x1080?',
    'artist': [{'name': 'Nathanial Mendoza',
      'email': 'whose1952@protonmail.com',
      'phone': '0115 843 7342'}]}],
  'product_name': 'february stages',
  'price': 14080.75,
  'order_date': datetime.datetime(2023, 5, 16, 20, 19, 12, 377000)},
 {'person': [{'name': 'Minda Stein',
    'email': 'increases2085@example.com',
    'phone': '0856 856 8728'}],
  'product': [{'description': 'satisfied oxford compatible calculated api but august specification rome keep',
    'category': 'pastel',
    'image_url': 'https://source.unsplash.com/1920x1080?',
    'artist': [{'name': 'Saul Elliott',
      'email': 'kingdom2068@gmail.com',
      'phone': '0996 093 3114'}

 ### Q3: Name and email for all customers in England

In [ ]:
%%time
person.create_index({"address.country": 1})

In [ ]:
%%timeit
list(person.find(
	{ "address.country": "England" },
	{ "_id": 0, "name": 1, "email": 1 }
))


 ### Q4: Count the number of confirmed orders in Q1 by artists in England

In [13]:
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
            "order_status": { "$in": ['delivered', 'processing', 'shipped'] },
            "$expr": { "$lt": ["$order_date", datetime(2023, 4, 1, 0, 0, 0, 0)]},
			"product.artist.address.country": "England",
		}
	},
	{ "$count": "count" }
]))


[{'count': 391}]

In [ ]:
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 0, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
			"product.artist.address.country": "England"
		}
	},
	{ "$count": "count" }
]))

In [ ]:
list(order.aggregate([
	{
		"$lookup": {
			"from": "product",
			"localField": "product",
			"foreignField": "_id",
			"pipeline": [
				{
					"$lookup": {
						"from": "artist",
						"localField": "artist",
						"foreignField": "_id",
						"pipeline": [
							{ "$project": { "_id": 1, "address.country": 1 } }
						],
						"as": "artist",
					}
				},
				{ "$project": { "_id": 0, "artist": 1 } }
			],
			"as": "product",
		}
	},
	{
		"$match": {
			"product.artist.address.country": "England"
		}
	},
	{ "$project": { "_id": 0, "product.artist._id": 1} }
]))

In [ ]:
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 0, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
			"artist.address.country": "England"
		}
	},
	{ "$count": "count" }
]))

In [ ]:
product.distinct("artist")

In [ ]:
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
			"artist.address.country": "England"
		}
	},
	{ "$project": { "_id": 0, "artist._id": 1} }
]))

In [ ]:
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{ "$project": { "_id": 0, "artist._id": 1, "artist.address.country":1} }
]))

In [ ]:
sourceFile = open('demo.txt', 'w')
print(
list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
			"artist.address.country": "England"
		}
	},
	{ "$project": { "_id": 0, "artist._id": 1} }
]))
, file = sourceFile)
sourceFile.close()

In [ ]:
sourceFile = open('demo.txt', 'w')
print('Hello, Python!', file = sourceFile)
sourceFile.close()

In [ ]:
len(list(product.aggregate([
        {
			"$lookup": {
				"from": "artist",
				"localField": "artist",
				"foreignField": "_id",
				"pipeline": [
					{ "$project": { "_id": 1, "name": 1, "address.country": 1 } }
				],
				"as": "artist",
			}
		},
	{
		"$match": {
            "artist._id": { "$nin": SDB_artists }
		}
	},
	{ "$project": { "_id": 1, "artist": 1} }
])))

In [ ]:
product.count_documents({
})

In [ ]:
artist.count_documents({
	"address.country": "England"
})

In [ ]:
order.distinct("order_status", {"$and": [{ "order_status": { "$ne": None } }, { "order_status": { "$ne": "pending" } }]})

In [ ]:
order.distinct("order_status", {"order_status": { "$nin": ["pending", None] }})

In [ ]:
%%timeit
order.count_documents({
    "$expr": { "$lte": [{ "$month": "$order_date" }, 3] },
	"order_status": { "$nin": ["pending", None] }
})

In [ ]:
order.create_index({"order_date": 1, "order_status": 1})

In [ ]:
%%timeit
order.count_documents({
	"order_status": { "$in": ['delivered', 'processing', 'shipped'] },
	"$expr": { "$lte": [{ "$month": "$order_date" }, 3] }
})

In [ ]:
%%timeit
order.count_documents({
	"order_status": { "$in": ['delivered', 'processing', 'shipped'] },
	"$expr": { "$lt": ["$order_date", datetime(2023, 4, 1, 0, 0, 0, 0)]}
})

In [ ]:
%%timeit
order.estimated_document_count({
	"order_status": { "$in": ['delivered', 'processing', 'shipped'] },
	"$expr": { "$lt": ["$order_date", datetime(2023, 4, 1, 0, 0, 0, 0)]}
})

In [ ]:
order.count_documents({
	"$and": [{ "order_status": { "$ne": None } }, { "order_status": { "$ne": "pending" } }]
})

In [ ]:
order.count_documents({
	"$expr": { "$lte": [{ "$month": "$order_date" }, 3] }
})

In [ ]:
list(order.find(
	{"$expr": { "$lte": [{ "$month": "$order_date" }, 3] }},
	{ "_id":0, "order_date": { "$month": "$order_date" } }
))

 ### Q5: Delete a specific review

In [ ]:
%%time
review.delete_one({ "_id": review_ids[0] })


 ### Q6: Delete reviews from a particular category

In [ ]:
%%timeit
product.distinct("_id", { "category": "charcoal" })

In [ ]:
%%timeit
product.distinct("_id", { "category": "charcoal" })

In [ ]:
%%time
review.delete_many({ "product": { "$in": product.distinct("_id", { "category": "charcoal" }) } })


 ### Q7: Update a customer address

In [ ]:
%%timeit
person.update_one(
	{ "_id": person_ids[randint(0, person_id_count)] },
	{
		"$set": {
			"address": {
				'address_line_1': '497 Ballycander',
				'address_line_2': "null",
				'city': 'Bromyard',
				'country': 'Wales',
				'post_code': 'ZX8N 4VJ',
				'coordinates': [68.772592, -35.491877]
			}
		}
	}
)


 ### Q8: Update discounts for products

In [ ]:
product.create_index({"price": 1})

In [ ]:
%%time
product.update_many(
	{ "price": { "$lt": 1000 } },
	{ "$set": { "discount": 0.2 } }
)

 ### Q9: "Transaction"* order from a new customer

In [ ]:
%%timeit
# Transaction - order from a new customer

new_person_id = uuid4()
random_product_id = product_ids[randint(0, product_id_count)]

# insert into the person table
person.insert_one({
		'_id': new_person_id,
		'first_name': 'Karyl',
		'last_name': 'Langley',
		'name': 'Karyl Langley',
		'company_name': "null",
		'email': 'dee1961@gmail.com',
		'phone': '+44 47 3516 5895',
		'address': {
			'address_line_1': '510 Henalta',
			'address_line_2': "null",
			'city': 'Lyme Regis',
			'country': 'Northern Ireland',
			'post_code': 'TO6Q 8CM',
			'coordinates': [-34.345071, 118.564172]
		}
	})

# relate into the order table
order.insert_one({
	"_id": uuid4(),
	"person": new_person_id,
	"product": random_product_id,
	'currency': '£',
	'discount': db.product.distinct("discount", { "_id" : random_product_id }), 
	"order_date": datetime.now(tz=timezone.utc),
	"order_status": "pending",
	"payment_method": "PayPal",
	"price": db.product.distinct("price", { "_id" : random_product_id }),
	"product_name": db.product.distinct("name", { "_id" : random_product_id }),
	"quantity": 1,
	"shipping_address": db.person.distinct("address", { "_id" : new_person_id })
})

# update the product table to reduce the quantity
product.update_one(
	{ "_id": random_product_id },
	{ "$inc": { "quantity": -1 } }
)


 ### Q10: "Transaction"* - New Artist creates their first product

In [ ]:
%%timeit
# Transaction - New Artist creates their first product

new_artist_id = uuid4()

# insert into the artist table
artist.insert_one({
    '_id': new_artist_id,
    'first_name': 'Anderson',
    'last_name': 'West',
    'name': 'Anderson West',
    'company_name': 'Atkins(ws) (ATK)',
    'email': 'six1933@gmail.com',
    'phone': '056 5881 1126',
    'address': {
        'address_line_1': '639 Connaugh',
        'address_line_2': "null",
        'city': 'Ripon',
        'country': 'Scotland',
        'post_code': 'CG3U 4TH',
        'coordinates': [4.273648, -112.907273]
	}
})

# insert into the product table
product.insert_one({
    '_id': uuid4(),
    'name': 'managed edt allocated pda',
    'description': 'counseling dildo greek pan works interest xhtml wrong dennis available cl specific next tower webcam peace magic',
    'category': 'watercolor',
    'price': 15735.96,
    'currency': '£',
    'discount': "null",
    'quantity': 1,
    'image_url': 'https://source.unsplash.com/1920x1080?',
    "artist": new_artist_id,
    "creation_history": {
        "quantity": 1,
        "created_at": datetime.now(tz=timezone.utc)
	}
})

In [ ]:
client.close()